Reference - this code is just modification of https://github.com/dennybritz/reinforcement-learning

In [3]:
%run ../utils/commonImports.py
%run ../utils/tradingImports.py
%matplotlib inline

In [4]:
import gym
import itertools
import matplotlib
import sys
import tensorflow as tf
import collections
from copy import deepcopy

matplotlib.style.use('ggplot')

# Load Data

In [5]:
dataRoot = 'D:\\Dropbox\\My work\\krypl-project\\dataLabeled\\poloniex\\5min'
fileName = 'USDT_BTC_5min_2016-01-01_2017-12-31.tsv'
file = '{dataRoot}\\{fileName}'.format(dataRoot=dataRoot, fileName=fileName)

usdtBtc = pd.read_csv(file, sep='\t').query("date >= '2017-01-01'")\
    .sort_values('timestamp').reset_index().drop('index', axis=1)
usdtBtc = usdtBtc

trainRatio = 0.7
trainSize = int(usdtBtc.shape[0] * trainRatio)
usdtBtcTrain = usdtBtc.iloc[:trainSize]
usdtBtcTest = usdtBtc.iloc[trainSize:].reset_index().drop('index', axis=1)

usdtBtcManagerTrain = CurrencyDataManager(usdtBtcTrain)
usdtBtcManagerTest = CurrencyDataManager(usdtBtcTest)
wallet = {'usdt': 1000}
contractPair = ContractPair('usdt', 'btc')

In [9]:
%%timeit
for i in range(1000):
    usdtBtc.iloc[10:11, :]

1 loop, best of 3: 250 ms per loop


In [15]:
%%timeit
for i in range(1000):
    usdtBtc.ix[10:11, :]

1 loop, best of 3: 276 ms per loop


C:\Anaconda3\lib\site-packages\ipykernel\__main__.py:258: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated


In [14]:
%%timeit
arr = usdtBtc.values
for i in range(1000):
    arr[10:11, :]

1 loop, best of 3: 121 ms per loop
